In [12]:
import os
import bs4
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain_chroma import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders import PyPDFLoader, WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

load_dotenv()

api_key = os.getenv('GROQ_API_KEY')

In [2]:
def load_llm():
    llm = ChatGroq(groq_api_key=api_key, model_name="llama3-8b-8192", temperature=0)
    return llm

In [3]:
def prepare_retriever():
    # loader = PyPDFLoader('pdf\Attention-is-all-you-need.pdf')
    loader = WebBaseLoader(
        web_paths=("https://www.nature.com/articles/s41467-020-16278-6",),
        bs_kwargs=dict(
            parse_only=bs4.SoupStrainer(
                class_=("c-article-title", "c-article-section__content")
            )
        ),
    )
    documents = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=64)
    texts_chunks = text_splitter.split_documents(documents)

    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2", model_kwargs={"device": "cpu"}, encode_kwargs={"normalize_embeddings": True},)
    
    db = Chroma.from_documents(documents=texts_chunks, embedding=embeddings)
    retriever = db.as_retriever()
    return retriever

In [4]:
def generate_history_aware_retriever(llm, retriever):
    contextualize_q_system_prompt = (
        'Taking into account the chat history and the latest user question that may be referencing the chat history,'
        'generate a new question that can be understood without the chat history. DO NOT answer that question,'
        'just reformulate it if needed and otherwise return it as is.'
    )
    contextualize_q_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", contextualize_q_system_prompt),
            MessagesPlaceholder("chat_history"),
            ("human", "{input}"),
        ]
    )
    history_aware_retriever = create_history_aware_retriever(
        llm, retriever, contextualize_q_prompt
    )
    return history_aware_retriever

In [5]:
def create_qa_chain(llm, history_aware_retriever):
    system_prompt = (
        'You are a helpful assistant that answers questions.'
        'Use the retrieved context to answer the question.'
        'If you do not know the answer your reply should be "I dont know."'
        'Try to keep the answers short unless otherwise specifed by the question.'
        '\n\n'
        '{context}'
        )
    qa_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt),
            MessagesPlaceholder("chat_history"),
            ("human", "{input}"),
        ]
    )
    question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

    qa_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)
    return qa_chain

In [6]:
store = {}

llm = load_llm()
retriever = prepare_retriever()
history_aware_retriever = generate_history_aware_retriever(llm, retriever)
rag_chain = create_qa_chain(llm, history_aware_retriever)

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [7]:
conversational_rag_chain.invoke(
    {"input": "What is confirmation bias?"},
    config={
        "configurable": {"session_id": "abc123"}
    },
)["answer"]

"According to the article, confirmation bias refers to the tendency of the brain to selectively accumulate new information that supports one's initial decision or choice, and to discount or ignore information that contradicts it. This bias is driven by confidence in one's initial decision, and is more pronounced when individuals are highly confident in their choice."

In [8]:
conversational_rag_chain.invoke(
    {"input": "What are some negative consequences of the above fact?"},
    config={"configurable": {"session_id": "abc123"}},
)["answer"]

'According to the article, in the current task where new evidence is always helpful, this bias against incorporating conflicting post-decision evidence is normatively maladaptive. This means that it can lead to:\n\n* Ignoring or discounting important information that contradicts one\'s initial decision\n* Making decisions that are less accurate or informed\n* Being "blind" to disconfirmatory evidence\n\nIn other scenarios where new evidence may be distracting or actively misleading, this confirmation bias might actually be helpful. However, in situations where new evidence is important for making accurate decisions, this bias can have negative consequences.'

In [9]:
conversational_rag_chain.invoke(
    {"input": "How was MEG recorded?"},
    config={"configurable": {"session_id": "abc123"}},
)["answer"]

'According to the article, MEG was recorded continuously at 600 samples/second using a whole-head 273-channel axial gradiometer system (CTF Omega, VSM MedTech).'

In [10]:
conversational_rag_chain.invoke(
    {"input": "What was my previous question?"},
    config={"configurable": {"session_id": "abc123"}},
)["answer"]

'Your previous question was "How was MEG recorded?"'

In [11]:
store

{'abc123': InMemoryChatMessageHistory(messages=[HumanMessage(content='What is confirmation bias?'), AIMessage(content="According to the article, confirmation bias refers to the tendency of the brain to selectively accumulate new information that supports one's initial decision or choice, and to discount or ignore information that contradicts it. This bias is driven by confidence in one's initial decision, and is more pronounced when individuals are highly confident in their choice."), HumanMessage(content='What are some negative consequences of the above fact?'), AIMessage(content='According to the article, in the current task where new evidence is always helpful, this bias against incorporating conflicting post-decision evidence is normatively maladaptive. This means that it can lead to:\n\n* Ignoring or discounting important information that contradicts one\'s initial decision\n* Making decisions that are less accurate or informed\n* Being "blind" to disconfirmatory evidence\n\nIn ot